In [ ]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd

# Correctly load the data from CSV
df = pd.read_csv("CleanedData/player_stats.csv")  

# Build App
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col(html.Div([
                    dbc.Card(
                        dbc.CardBody([
                            html.Div([
                                html.H2("Select Players"),
                            ], style={'textAlign': 'center'})
                        ])
                    ),
                ]), width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='player-dropdown',
                        options=[{'label': i, 'value': i} for i in df['player'].unique()],  
                        multi=True,
                        placeholder="Select players",
                        style={"color": "#000"}
                    )
                ], width=12),
            ], align='center'),
            html.Br(),
            dbc.Row([
                dbc.Col(dcc.Graph(id='stacked-bar-chart'), width=12),
            ], align='center'),
        ]), color='dark'
    )
])

@app.callback(
    Output('stacked-bar-chart', 'figure'),
    [Input('player-dropdown', 'value')]
)
def update_graph(selected_players):
    if selected_players:
        fig = go.Figure()
        for i, player in enumerate(selected_players):
            filtered_df = df[df['player'] == player]
            goals_sum = filtered_df['goals'].sum()
            assists_sum = filtered_df['assists'].sum()
            
            # Only show the legend for the first player to avoid repetition
            show_legend = True if i == 0 else False
            
            fig.add_trace(go.Bar(
                name='Goals',
                x=[player],
                y=[goals_sum],
                marker_color='red',  # Change colour 
                showlegend=show_legend  # Control legend display
            ))
            fig.add_trace(go.Bar(
                name='Assists',
                x=[player],
                y=[assists_sum],
                marker_color='blue',  # Change colour 
                showlegend=show_legend  # Control legend display
            ))

        fig.update_layout(
            barmode='stack',
            title='Player Goals and Assists',
            xaxis_title='Player',
            yaxis_title='Count',
            legend_title='Metric'
        )
        return fig
    return go.Figure()

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
